## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
from lonelyboy.geospatial import group_patterns_v2 as gsgp

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

In [2]:
import psycopg2
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point, LineString, shape
from haversine import haversine
from datetime import datetime, timedelta

In [3]:
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime

## Import Libraries for Visualizations

In [4]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')

##  Importing the Server Credentials & Connectiing to Server and Fetch 48hrs of Trajectory Data
## Note: The Data Server is broken; Use the "subseted.pckl" file (code below)

In [5]:
traj = pd.read_pickle('data/pkl/traj_raw.pkl')
ports = pd.read_pickle('data/pkl/ports_raw.pkl')

In [6]:
%%time
properties = configparser.ConfigParser()
properties.read(os.path.join('.','sql_server.ini'))
properties = properties['SERVER']

host    = properties['host']
db_name = properties['db_name']
uname   = properties['uname']
pw      = properties['pw']
port    = properties['port']

traj_sql = 'SELECT * FROM ais_data.dynamic_ships WHERE ts>1456802710 AND ts<1456975510  '
ports_sql = 'SELECT * FROM ports.ports_of_brittany'

print (f'Connecting with {host}:{port}/{db_name}')
con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)

traj = gpd.GeoDataFrame.from_postgis(traj_sql, con, geom_col='geom' )

ports = gpd.GeoDataFrame.from_postgis(ports_sql, con, geom_col='geom' )
ports.geom = ports.geom.apply(lambda x: x[0])

print(f'Fetched {traj.memory_usage().sum()} bytes')
print(f'Fetched {ports.memory_usage().sum()} bytes')

con.close()

# SAVE IF NOT FETCHED
# pd.to_pickle(traj, 'data/traj_raw.pkl')
# pd.to_pickle(ports, 'data/ports_raw.pkl')

KeyError: 'SERVER'

In [7]:
ports.head(2)

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POINT (-1.771798868659233 48.60274269672541)
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POINT (-2.001990119062326 48.48369993456267)


In [8]:
traj.head(2)

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,profile_id
0,12293630,227941000,7.0,0.0,0.0,285.0,8,-4.327213,48.100086,1456802711,POINT (-4.3272133 48.100086),99990203
1,12293675,228186700,15.0,-127.0,102.3,360.0,511,-4.512577,48.370872,1456802766,POINT (-4.5125766 48.370872),99990203


## Some Preprocessing...

In [9]:
#### DROP TIMESTAMP DUPLICATES PER MMSI
traj = traj.drop_duplicates(subset=['mmsi', 'ts']).sort_values('ts').reset_index(drop=True)
traj['date'] = traj.ts.apply(int).apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
traj['month'] = traj['date'].apply(lambda x: x.split('-')[1])
traj['year'] = traj['date'].apply(lambda x: x.split('-')[0])

### CALCULATE VELOCITIES BASED ON THE POINTS
# traj['velocity'] = np.nan
# traj = traj.groupby(['mmsi'], as_index=False, group_keys=False).apply(gspp.calculate_velocity, smoothing=False, window=15, center=False)

### DENOISE SAMPLE_TRAJECTORIES BASED ON A VELOCITY THRESHOLD (POTENTIAL-AREA-OF-ACTIVITY)
# traj = gspp.PotentialAreaOfActivity(traj, velocity_threshold = 102.2)
traj.head()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,profile_id,date,month,year
0,12293630,227941000,7.0,0.0,0.0,285.0,8,-4.327213,48.100086,1456802711,POINT (-4.3272133 48.100086),99990203,2016-03-01,03,2016
1,12293631,227705102,15.0,-127.0,0.0,261.8,511,-4.496568,48.382435,1456802711,POINT (-4.496568 48.382435),99990203,2016-03-01,03,2016
2,17515087,256494000,5.0,0.0,0.0,344.0,217,-4.451149,48.383625,1456802713,POINT (-4.4511485 48.383625),99990203,2016-03-01,03,2016
3,12293634,227574020,15.0,-127.0,0.0,241.7,511,-4.496673,48.382454,1456802713,POINT (-4.496673 48.382454),99990203,2016-03-01,03,2016
4,17515086,227300000,7.0,-126.0,2.8,34.2,346,-4.631805,48.111330,1456802713,POINT (-4.631805 48.11133),99990203,2016-03-01,03,2016


In [17]:
gsplt.map_plot(traj.loc[traj.date=='2016-03-01'], ports, color=['darkblue', 'crimson'], markersize=25, figsize=(20,15))

## Computing Primitive Statistics
### __NOTE: The below code works as a proof of concept due to the fact that the subset dataframe does not statistically represent the distribution of the whole dataset.__

In [23]:
ports.head()

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POINT (-1.771798868659233 48.60274269672541)
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POINT (-2.001990119062326 48.48369993456267)
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POINT (-4.341204251638414 48.09709590770091)
3,4,port.101,101,Brézellec,29028,79105.4,2364190.0,POINT (-4.661115947908725 48.06804110561076)
4,5,port.102,102,Sein,29083,64562.5,2362180.0,POINT (-4.852944548180974 48.03825273921113)


* #### #Vessels

In [10]:
traj.mmsi.unique().size

197

* #### #Records per Day/Month/MMSI

In [57]:
# traj.groupby('date').id.count().plot.bar(figsize=(7,7), cmap='tab20', rot=0)    # PER DATE  
# traj.groupby(['year', 'month']).id.count().plot.bar(figsize=(7,7), cmap='tab20', rot=0)    # PER MONTH  
# traj.groupby('mmsi').id.count().plot.bar(figsize=(20,10), title='#Records per Vessel', cmap='tab20', fontsize=5.5)    # PER MMSI 
# plt.xlabel('mmsi')

# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_3.png'))

1136.8324873096446

* #### Average Records per Date/Month/MMSI

In [88]:
# The Averages
# traj.groupby('date').id.count().mean()             # PER DATE
# traj.groupby(['year', 'month']).id.count().mean()  # PER MONTH  
# traj.groupby('mmsi').id.count().mean().round()     # PER MMSI 
traj.groupby(['date', 'mmsi']).id.count().groupby(['date']).mean().round().plot.bar(title='Avg. Records per Vessel per Day', cmap='tab20', fontsize=10, rot=0)     # PER MMSI PER DAY  
# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_3.png'))

* #### Average Vessel Speed (in knots) per Day/Month

In [92]:
# ax = traj.groupby('date').velocity.mean().plot.bar(rot=0, title='Avg. Velocity (knots) per Day', color='steelblue')
ax = traj.groupby('date').speed.mean().plot.bar(rot=0, title='Avg. Velocity (knots) per Day', color='steelblue')
# traj.groupby(['year','month']).speed.mean().plot.bar(rot=0, title='Avg. Velocity (knots) per Month', color='steelblue')
ax.set_ylabel('Velocity (knots)')
# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_4.png'))

Text(0, 0.5, 'Velocity (knots)')

* #### Accelation Distribution

In [93]:
window=150
pd.cut(traj.groupby('mmsi').speed.rolling(window).mean().diff().rolling(window).mean(), 200).value_counts(sort=False).plot.area(figsize=(12,10), cmap='tab20', title='Vessel Acceleration Distribution', rot=8)
# pd.cut(traj.groupby('mmsi').velocity.rolling(window).mean().diff().rolling(window).mean(), 200).value_counts(sort=False).plot.area(figsize=(12,10), cmap='tab20', title='Vessel Acceleration Distribution', rot=8)
# pd.cut(traj.groupby('mmsi').speed.diff().ffill().bfill(), 200).value_counts(sort=False).plot.area(figsize=(12,10), cmap='tab20', title='Vessel Acceleration Distribution', rot=8)
plt.xlabel('Acceleration')
# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_5.png'))

Text(0.5, 0, 'Acceleration')

* #### Time Spent in/out of Port

In [94]:
%%time
ports_epsg_3310 = ports.to_crs(epsg=3310,inplace=False)
pnt_port_dist = traj.to_crs(epsg=3310,inplace=False).apply(lambda row: gspp.distance_to_nearest_port(row.geom, ports_epsg_3310), axis=1)

CPU times: user 4min, sys: 145 ms, total: 4min
Wall time: 4min


In [14]:
inside_port = pnt_port_dist.loc[pnt_port_dist/1000 <= 5].index
outside_port = pnt_port_dist.loc[pnt_port_dist/1000 > 5].index

# traj.loc[inside_port].groupby(['date', 'mmsi']).apply(lambda x: x.ts.diff().sum()/3600).to_frame()
hrs_inside_port = traj.loc[inside_port].groupby(['date', 'mmsi']).apply(lambda x: x.ts.diff().sum()/3600).to_frame().reset_index().rename({0:'hrs'}, axis=1).groupby('date').apply(lambda x: x.hrs.sum())
hrs_inside_port

# traj.loc[outside_port].groupby(['date', 'mmsi']).apply(lambda x: x.ts.diff().sum()/3600).to_frame()
hrs_outside_port = traj.loc[outside_port].groupby(['date', 'mmsi']).apply(lambda x: x.ts.diff().sum()/3600).to_frame().reset_index().rename({0:'hrs'}, axis=1).groupby('date').apply(lambda x: x.hrs.sum())
hrs_outside_port

pd.concat([hrs_inside_port.rename('Inside Port (hrs)'), hrs_outside_port.rename('Outside Port (hrs)')],
         axis=1).plot.bar(rot=0, title='Cumulative time per Vessel Inside & Outside of Port', cmap='tab20')
# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_6.png'))

* #### Common Vessel Destinations per Month

In [18]:
k=5
k_common_heading = traj.groupby(['year', 'month']).apply(lambda x: x.heading.value_counts().to_frame().iloc[0:k]).reset_index()
k_common_heading.rename({'level_2': 'heading', 'heading': 'num_visits'}, axis=1, inplace=True)
k_common_heading.set_index(['year', 'month', 'heading'], inplace=True)
ax = k_common_heading.plot.bar(y='num_visits', color='steelblue', title='Common Vessel Destinations per Month', rot=20)
# plt.savefig(os.path.join(os.path.expanduser('~'),'Desktop', 'Figure_7.png'))

## Vessel-Oriented Data Exploratory Statistics

In [25]:
vessels = pd.read_csv('data/csv/EuropeanVesselRegister.csv', sep=';')
vessels.head()

/home/andretri7/anaconda3/envs/gis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (29,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,countrycode,cfr,eventcode,eventstartdate,eventenddate,licenseind,registrationnbr,extmarking,vesselname,portcode,...,commonth,comday,segment,expcountry,exptype,publicaidcode,decisiondate,decisionsegcode,constructionyear,constructionplace
0,BEL,BEL000021964,CEN,19890101,19910417,N,002,A.2,NANCY,ANTWE,...,3,26,BIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BEL,BEL000021964,MOD,19910418,19911231,N,002,A.2,NANCY,ANTWE,...,3,26,BIN,NaN,EX,NaN,NaN,NaN,NaN,NaN
2,BEL,BEL000021964,MOD,19920101,19940728,Y,002,O.2,NANCY,OOSTE,...,3,26,E10,NaN,EX,NaN,NaN,NaN,NaN,NaN
3,BEL,BEL000021964,MOD,19940729,19961231,Y,002,O.2,NANCY,OOSTE,...,3,26,C10,NaN,EX,NaN,NaN,NaN,NaN,NaN
4,BEL,BEL000021964,MOD,19970101,20021231,Y,002,O.2,NANCY,OOSTE,...,3,26,4A1,NaN,EX,NaN,NaN,NaN,1964.0,"N.V. Scheepswerf BAREVELD, Wildervank, NL"


In [26]:
len(vessels)

487126

In [8]:
ves_gear = pd.read_csv('data/table_gear_type_code.csv', sep=',', names=['gearcode', 'geardesc', 'gearcat'])
ves_gear.head()

,gearcode,geardesc,gearcat
0,PS,With purse lines (purse seines),HARVESTING MACHINES
1,PS1,one boat operated purse seines,HARVESTING MACHINES
2,PS2,two boats operated purse seines,HARVESTING MACHINES
3,LA,Without purse lines (lampara),HARVESTING MACHINES
4,SB,Beach seines,SEINE NETS


* #### Histogram regarding the vessels country of origin

In [9]:
plt.figure(1)
vessels.countrycode.value_counts(ascending=True).plot('barh', color='steelblue')

* #### Histogram regarding the vessels main gear types

In [10]:
plt.figure(2)
# vessels.gearmaincode.value_counts().plot('bar')
ves_gear.merge(vessels.gearmaincode.to_frame(), left_on='gearcode', right_on='gearmaincode').geardesc.value_counts(ascending=True).plot('barh', rot=40, color='steelblue') 

* #### Histogram regarding the vessels main gear category

In [11]:
plt.figure(3)
# vessels.gearmaincode.value_counts().plot('bar')
ves_gear.merge(vessels.gearmaincode.to_frame(), left_on='gearcode', right_on='gearmaincode').gearcat.value_counts(ascending=True).plot('barh', rot=25, color='steelblue') 

## OLD (and maybe useless) STATISTICS

* #### Average Speed (in knots) per MMSI

In [68]:
ax = traj.groupby('mmsi').velocity.mean().sort_values().plot.barh(rot=0, color='steelblue', fontsize=10)
ax = traj.groupby('mmsi').speed.mean().sort_values().plot.barh(rot=0, color='steelblue', fontsize=10)
ax.set_xlabel('Velocity (knots)')

* #### Vessel Velocity Plot Through Time (Spaghetti Plot)

In [67]:
fig, ax = plt.subplots(figsize=(15,7))
palette = plt.get_cmap('tab20c')
traj.groupby('mmsi').velocity.apply(lambda src: src.plot(rot=0, fontsize=10, color=palette((num)%20), ax=ax))
plt.ylabel('Velocity (knots)')

# libraries and data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# style
plt.style.use('seaborn-darkgrid')
# create a color palette
palette = plt.get_cmap('tab20c')
# multiple line plot
num=0
for mmsi_name in traj.groupby('mmsi').groups:
    num+=1
#     plt.plot(traj.loc[traj.mmsi == mmsi_name].ts, traj.loc[traj.mmsi == mmsi_name].velocity, marker='', color=palette(num%20), linewidth=1, alpha=0.9)
    plt.plot(traj.loc[traj.mmsi == mmsi_name].ts, traj.loc[traj.mmsi == mmsi_name].speed, marker='', color=palette(num%20), linewidth=1, alpha=0.9)
# Add titles
plt.tight_layout()
plt.title("Vessel Velocity Plot Through Time", loc='center', fontsize=12, fontweight=0)
plt.savefig("Figure_223_cand.png", bbox_inches = 'tight')

In [ ]:
# PREVIOUS (BIT USELESS CODE) FOR COMMON DESTINATIONS PER MONTH
k_common_heading = traj.groupby(['year', 'month']).apply(lambda x: pd.Series(x.heading.value_counts().index[0:3]).to_frame())
k_common_heading = k_common_heading.reset_index().drop('level_2', axis=1).set_index(['year', 'month'])
k_common_heading.rename({0: 'heading'}, axis=1).plot.bar(rot=0, color='dodgerblue')

traj.groupby(['year', 'month']).apply(lambda x: x.heading.value_counts().index[0:3]).reset_index()
traj.groupby(['year', 'month']).apply(lambda x: x.heading.value_counts().index[0:3].to_frame()).reset_index(inplace=False).drop(0, axis=1)

In [12]:
# TESTING CODE FOR HAVERSINE DISTANCE IN EPSG 3310
import geopandas as gpd
from shapely.geometry import Point,Polygon
geom=[Point(xy) for xy in zip([117.454361,117.459880],[38.8459879,38.846255])]
gdf=gpd.GeoDataFrame(geometry=geom,crs={'init':'epsg:4326'})
gdf.to_crs(epsg=3310,inplace=True)
l=gdf.distance(gdf.shift())
print(l)

ports.to_crs(epsg=3310,inplace=False).geom.distance(traj.to_crs(epsg=3310,inplace=False).geom[0])/1000